# Import packages

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import mean_squared_error, mean_absolute_error, \
    r2_score, make_scorer, recall_score, accuracy_score, f1_score, \
    precision_score, balanced_accuracy_score, roc_curve, auc

# Custom functions

In [3]:
from pickle_managment import save_pickle, load_pickle

# Regression

In [3]:
regression_df_expanded_cleaned = pd.read_csv(
    r'datasets\train_datasets\regression_df_expanded_cleaned_train.csv.zip'
)
regression_df_expanded_cleaned

,SMILES,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,...,158_y,159_y,160_y,161_y,162_y,163_y,164_y,165_y,166_y,logBB
0,CC(C)(C)OC(=O)CCCC1=CC=C(C=C1)N(CCCl)CCCl,11.682268,11.682268,0.134704,-0.409691,0.474821,11.347826,360.325,333.109,359.141884,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.00
1,Clc1ccc(N2)c(C(C)=NC(O)C2=O)c1,11.306112,11.306112,0.532016,-1.350763,0.700249,20.066667,224.647,215.575,224.035255,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.61
2,CC1=C2OC3=C(C=CC=C3)C3(O)CNCC3C2=CC=C1,11.238358,11.238358,0.028750,-0.893218,0.770818,27.000000,267.328,250.192,267.125929,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.39
3,C[C@H]1[C@H]([C@H](C[C@@H](O1)O[C@H]2C[C@@](CC...,13.643057,13.643057,0.017252,-2.240737,0.235263,29.846154,543.525,514.293,543.174061,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-0.83
4,CCC1(C(=O)NC(=O)NC1=O)C2=CC=CC=C2,11.975791,11.975791,0.293611,-1.310347,0.736884,18.529412,232.239,220.143,232.084792,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,C(=O)(N)N,9.000000,9.000000,0.833333,-0.833333,0.370507,6.000000,60.056,56.024,60.032363,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,-0.14
941,C[C@@H](CC1=CC=CC=C1)NC,3.213399,3.213399,0.567037,0.567037,0.691109,12.909091,149.237,134.117,149.120449,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.96
942,CNC(=O)C1=C(N=C(N=C1OCC2CCN(CC2)C)C#N)NCC3CCC4...,12.730831,12.730831,0.023700,-0.298674,0.670950,21.531250,440.592,404.304,440.289974,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.37
943,CCC1=C2C(=CC=C1)C3=C(N2)C(OCC3)(CC)CC(=O)O,11.285827,11.285827,0.004742,-0.820305,0.905784,21.428571,287.359,266.191,287.152144,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-1.42


In [4]:
regression_X = regression_df_expanded_cleaned.loc[
               :,
               ~regression_df_expanded_cleaned.columns.isin(
                   ['SMILES', 'logBB'])
               ]

regression_y = regression_df_expanded_cleaned['logBB']

# Use the transformer that the SVM regressor has used before
data_processing_pipeline =load_pickle(r'model_outputs\svm_regressor\svm_regressor_pipeline.pkl')
regression_X_processed=data_processing_pipeline.transform(regression_X)
regression_X_processed

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,157_y,158_y,159_y,160_y,161_y,162_y,163_y,164_y,165_y,166_y
0,0.417052,0.417240,-0.342126,0.166963,-0.675608,-0.788599,0.242281,0.219503,0.237570,0.161789,...,0.877304,0.470032,0.838274,0.614452,0.436092,0.485071,0.418776,0.511549,0.347845,-0.072932
1,0.309766,0.309953,0.397096,-0.486012,0.499964,0.214680,-0.590699,-0.554520,-0.592608,-0.665765,...,0.877304,0.470032,0.838274,0.614452,0.436092,0.485071,0.418776,0.511549,0.347845,-0.072932
2,0.290441,0.290628,-0.539258,-0.168538,0.867969,1.012500,-0.328663,-0.326549,-0.327833,-0.283817,...,0.877304,0.470032,0.838274,0.614452,0.436092,0.485071,0.418776,0.511549,0.347845,-0.072932
3,0.976308,0.976493,-0.560650,-1.103531,-1.924866,1.340008,1.367017,1.412695,1.368376,1.403120,...,0.877304,0.470032,0.838274,0.614452,0.436092,0.485071,0.418776,0.511549,0.347845,-0.072932
4,0.500771,0.500958,-0.046470,-0.457969,0.691009,0.037788,-0.544088,-0.524437,-0.543147,-0.506620,...,-1.139855,0.470032,0.838274,0.614452,0.436092,0.485071,0.418776,0.511549,0.347845,-0.072932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,-0.347982,-0.347793,0.957713,-0.126987,-1.219586,-1.403974,-1.601186,-1.605247,-1.600342,-1.525148,...,-1.139855,0.470032,-1.192928,-1.627467,0.436092,-2.061553,-2.387913,0.511549,-2.874840,-0.072932
941,-1.998434,-1.998239,0.462254,0.844677,0.452298,-0.608944,-1.053670,-1.090964,-1.052931,-0.952226,...,-1.139855,0.470032,-1.192928,0.614452,0.436092,0.485071,0.418776,-1.954847,0.347845,-0.072932
942,0.716123,0.716310,-0.548655,0.243993,0.347174,0.383210,0.735071,0.688359,0.736193,0.862027,...,0.877304,0.470032,0.838274,0.614452,0.436092,0.485071,0.418776,0.511549,0.347845,-0.072932
943,0.303980,0.304168,-0.583926,-0.117947,1.571798,0.371395,-0.205685,-0.221187,-0.204779,-0.124672,...,0.877304,-2.127514,0.838274,0.614452,0.436092,0.485071,0.418776,0.511549,0.347845,-0.072932


In [5]:
regression_X_train, regression_X_test, regression_y_train, regression_y_test = train_test_split(
    regression_X_processed,
    regression_y,
    test_size=0.2,
    random_state=1,
    shuffle=True
)

In [6]:
save_pickle(regression_X_train,
            r'model_outputs\rf_regressor\regression_X_train.pkl')

save_pickle(regression_y_train,
            r'model_outputs\rf_regressor\regression_y_train.pkl')

save_pickle(regression_X_test,
            r'model_outputs\rf_regressor\regression_X_test.pkl')

save_pickle(regression_y_test,
            r'model_outputs\rf_regressor\regression_y_test.pkl')

0

## Random forest

### Model training

In [7]:
start_time = datetime.now()

rf_regressor = RandomForestRegressor(
    random_state=1,
    bootstrap=True, #Whether bootstrap samples are used when building trees.
    # Default to be True
    oob_score=True #Whether to use out-of-bag samples to estimate the
    # generalization score. Only available if bootstrap=True
)

rf_regressor_grid_search = GridSearchCV(
    estimator=rf_regressor,
    param_grid={
        'n_estimators': [500, 1000],  #Number of trees
        'max_depth': [5, 20],  #The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples
        'min_samples_split': [2, 50], #The minimum number of samples required
        # to split an internal node. Default to be 2
        'min_samples_leaf': [5, 50], #The minimum number of samples required
        # to be at a leaf node. The branch will stop splitting once the
        # leaves have that number of samples each. Having 1 here is more
        # prone to overfitting
        'criterion': ['squared_error', 'absolute_error'], # “squared_error” for the mean
        # squared error is the default
        'max_samples':[0.2, 0.85],
        'max_features': ['sqrt'] #Reduce the number of features can
        # mitigate overfitting issue
    },
    cv=8,  #Number of fold for cross validation. It should be 8 or 10
    scoring={
        # All these are only viable in the negative option
        'MAE': 'neg_mean_absolute_error',
        'MSE': 'neg_mean_squared_error',
        'R2': 'r2'
    },
    refit='R2', #MAE is less sensitive to outliers and can help reduce
    # overfitting

    n_jobs=1,
    # -1 means using all processors, but it won't give you any messages.
    # Only using 1 for my computer print out the training messages

    verbose=10  #Provide detailed more messages
)

rf_regressor_grid_search.fit(regression_X_train, regression_y_train)

end_time = datetime.now()
print('GridSearchCV took {}'.format(end_time - start_time))

Fitting 8 folds for each of 64 candidates, totalling 512 fits
[CV 1/8; 1/64] START criterion=squared_error, max_depth=5, max_features=sqrt, max_samples=0.2, min_samples_leaf=5, min_samples_split=2, n_estimators=500
[CV 1/8; 1/64] END criterion=squared_error, max_depth=5, max_features=sqrt, max_samples=0.2, min_samples_leaf=5, min_samples_split=2, n_estimators=500; MAE: (test=-0.518) MSE: (test=-0.423) R2: (test=0.370) total time=   7.8s
[CV 2/8; 1/64] START criterion=squared_error, max_depth=5, max_features=sqrt, max_samples=0.2, min_samples_leaf=5, min_samples_split=2, n_estimators=500
[CV 2/8; 1/64] END criterion=squared_error, max_depth=5, max_features=sqrt, max_samples=0.2, min_samples_leaf=5, min_samples_split=2, n_estimators=500; MAE: (test=-0.464) MSE: (test=-0.351) R2: (test=0.396) total time=   8.5s
[CV 3/8; 1/64] START criterion=squared_error, max_depth=5, max_features=sqrt, max_samples=0.2, min_samples_leaf=5, min_samples_split=2, n_estimators=500
[CV 3/8; 1/64] END criterio

In [8]:
rf_regressor_results_df = pd.DataFrame(rf_regressor_grid_search.cv_results_)
#Make the GridSearch results into a df
rf_regressor_results_df.drop(
    list(rf_regressor_results_df.filter(regex='time|split|std')),
    axis=1,
    inplace=True
)  # Remove columns that aren't very interesting

rf_regressor_results_df = rf_regressor_results_df.sort_values(
    by='rank_test_R2')
rf_regressor_results_df

,param_criterion,param_max_depth,param_max_features,param_max_samples,param_min_samples_leaf,param_n_estimators,params,mean_test_MAE,rank_test_MAE,mean_test_MSE,rank_test_MSE,mean_test_R2,rank_test_R2
25,squared_error,20,sqrt,0.85,5,1000,"{'criterion': 'squared_error', 'max_depth': 20...",-0.409273,2,-0.294436,1,0.479227,1
24,squared_error,20,sqrt,0.85,5,500,"{'criterion': 'squared_error', 'max_depth': 20...",-0.409557,4,-0.294927,2,0.478393,2
57,absolute_error,20,sqrt,0.85,5,1000,"{'criterion': 'absolute_error', 'max_depth': 2...",-0.408393,1,-0.299963,3,0.469618,3
56,absolute_error,20,sqrt,0.85,5,500,"{'criterion': 'absolute_error', 'max_depth': 2...",-0.409451,3,-0.301412,4,0.466861,4
26,squared_error,20,sqrt,0.85,5,500,"{'criterion': 'squared_error', 'max_depth': 20...",-0.443840,7,-0.336551,5,0.406086,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,absolute_error,5,sqrt,0.2,50,500,"{'criterion': 'absolute_error', 'max_depth': 5...",-0.569206,49,-0.528010,57,0.071652,57
53,absolute_error,20,sqrt,0.2,50,1000,"{'criterion': 'absolute_error', 'max_depth': 2...",-0.569774,57,-0.528834,61,0.070352,61
39,absolute_error,5,sqrt,0.2,50,1000,"{'criterion': 'absolute_error', 'max_depth': 5...",-0.569774,57,-0.528834,61,0.070352,61
55,absolute_error,20,sqrt,0.2,50,1000,"{'criterion': 'absolute_error', 'max_depth': 2...",-0.569774,57,-0.528834,61,0.070352,61


In [9]:
rf_regressor_results_df.to_csv(
    r'model_grid_search\rf_regressor_results.csv',
    index=False
)

In [10]:
best_rf_regressor = rf_regressor_grid_search.best_estimator_
save_pickle(
    best_rf_regressor,
    r'model_pickles\best_rf_regressor.pkl'
)
# To load this best model again, use load_pickle(r'model_pickles\random_forest_regressor\best_rf_regressor.pkl')

0

# Classification

## Neural network
According to SVM classifiers' performance using the 2 methods of balancing
the dataset, the one created by centroid gave more reliable better results.
Consequently, that dataset is used to train the MLP classifier

In [4]:
classification_df_expanded_balanced = pd.read_csv(
    r'datasets\balanced_datasets\BBB_classification_balanced_centroid.csv.zip'
)
classification_df_expanded_balanced

,SMILES,BBB+/BBB-,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,...,157_y,158_y,159_y,160_y,161_y,162_y,163_y,164_y,165_y,166_y
0,CN[C@@H]1[C@H](O[C@H]2[C@H](O[C@@H]3[C@H](NC(=...,BBB+,9.095732,9.095732,0.282774,-0.017526,0.586761,11.860043,358.488,332.280,...,0.333333,1.0,0.666667,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,CCCCC[C@H](N)C(O)(c1ccccc1)c1ccccc1,BBB+,15.143070,15.143070,0.021937,-1.615734,0.147925,26.905882,1202.635,1090.747,...,1.000000,1.0,1.000000,1.0,1.0,0.0,0.0,1.0,1.0,0.0
2,Cc1nnc(SCC2=C(C(=O)O)N3C(=O)[C@@H](NC(=O)[C@H]...,BBB+,15.022344,15.022344,0.036460,-1.947471,0.109538,42.500000,1085.156,1008.548,...,1.000000,0.0,1.000000,1.0,0.0,1.0,1.0,1.0,1.0,0.0
3,C[C@@H]1C[C@H]2[C@@H]3C[C@H](F)C4=CC(=O)C(Cl)=...,BBB+,14.543654,14.543654,0.006968,-2.468650,0.116173,36.726027,1030.303,942.607,...,1.000000,1.0,1.000000,1.0,1.0,0.0,1.0,1.0,1.0,0.0
4,CC(C)=CCN1Cc2cc(Cl)cc3[nH]c(=S)n(c23)C[C@@H]1C,BBB+,14.995286,14.995286,0.035518,-1.944947,0.109538,42.500000,1085.156,1008.548,...,1.000000,0.0,1.000000,1.0,0.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5127,CC1CC2C(C(O)CC3(C)C2CCC3(O)C(=O)CO)C2(C)C=CC(=...,BBB-,12.428253,12.428253,0.083608,-1.721468,0.122643,47.948718,569.609,526.265,...,1.000000,1.0,1.000000,1.0,1.0,0.0,1.0,1.0,1.0,0.0
5128,CC[C@H]1CC(=O)[C@@H]2Oc3c(OC)ccc4c3[C@@]23CCN(...,BBB-,12.559895,12.559895,0.048057,-3.487690,0.443295,22.448276,440.503,420.343,...,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.0
5129,CN(C)C=Nc1c(I)cc(I)c(CCC(=O)O)c1I,BBB-,12.935121,12.935121,0.008292,-0.854188,0.192994,15.076923,551.769,498.345,...,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.0
5130,CC(=O)Oc1ccc(C(=C2CCCCC2)c2ccc(OC(C)=O)cc2)cc1,BBB-,13.042813,13.042813,0.190845,-3.798036,0.233814,31.275000,594.668,564.428,...,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [5]:
# Data pre-processing. This will make the model less interpretable but since
# MLP is already less interpretable than RF, this is fine since I will try
# to use another RF to explain why MLP made such decision

classification_X = classification_df_expanded_balanced.loc[
                   :,
                   ~classification_df_expanded_balanced.columns.isin(
                       ['SMILES', 'BBB+/BBB-'])
                   ]

classification_y = classification_df_expanded_balanced['BBB+/BBB-']


data_processing_pipeline =load_pickle\
    (r'model_outputs\svm_classifier\centroid_pipeline.pkl')

classification_X_processed=data_processing_pipeline.transform(classification_X)
classification_X_processed

array([[-5.21368953,  3.62439041,  3.83142447, ..., -0.28812032,
         1.01953803,  1.14219888],
       [26.17726014,  2.11795901,  7.38495038, ..., -0.06678346,
        -0.10646903,  0.11734033],
       [30.11439095, -6.29187033, 11.81692613, ...,  0.16951343,
         0.13249661, -0.20160932],
       ...,
       [ 4.95976758, -2.1562782 ,  4.54480143, ...,  0.19877777,
         0.478151  ,  0.5183374 ],
       [ 9.25061068,  9.77543853, -4.86349062, ...,  0.56208832,
        -0.43130308, -0.22326148],
       [-4.13634063,  2.39963095,  3.04815532, ..., -0.65332458,
         0.38452521,  0.0315058 ]])

In [6]:
classification_X_train, classification_X_test, classification_y_train, classification_y_test = train_test_split(
    classification_X_processed,
    classification_y,
    test_size=0.2,
    random_state=1,
    shuffle=True,
    stratify=classification_y #Ensure train set and test set have the same
    # ratio for the 2 categories
)

In [7]:
save_pickle(classification_X_train,
            r'model_outputs\mlp_classifier\classification_X_train.pkl')

save_pickle(classification_y_train,
            r'model_outputs\mlp_classifier\classification_y_train.pkl')

save_pickle(classification_X_test,
            r'model_outputs\mlp_classifier\classification_X_test.pkl')

save_pickle(classification_y_test,
            r'model_outputs\mlp_classifier\classification_y_test.pkl')

0

### Model training

In [8]:
start_time = datetime.now()

mlp_classifier = MLPClassifier(
    solver='adam',
    #"adam" is the default. It works pretty well on relatively large datasets (with thousands of training samples or more) in terms of both training time and validation score
    random_state=1,
    shuffle=True,  #shuffle samples in each iteration for "adam" solver
    early_stopping=True #Only effective when solver=’sgd’ or ‘adam’.
)

mlp_classifier_grid_search = GridSearchCV(
    estimator=mlp_classifier,
    param_grid={
        'hidden_layer_sizes': [(5,),(10,),(10,10)],
        'alpha': [1e-3, 1], #1e-4 is the default. Need stronger
        # regularization to avoid overfitting
        'activation': [
            'relu', #"relu" is the default
            'tanh'
        ],
        'batch_size': [16, 64], #Smaller batch size can help reducing
        # overfitting. Since the dataset has feature number > 200, the
        # default is to use 200 as the batch_size
        'learning_rate': ['constant', 'adaptive'] #Default to be "constant"
    },
    cv=RepeatedStratifiedKFold(
        n_splits=8,
        n_repeats=2, #Each time the split will be different
        random_state=1
    ),
    scoring={
        'Recall': make_scorer(
            recall_score, #Need pos_label
            pos_label='BBB+', #Without this, pos_label is default to be 1
            # and will through an error since 1 isn't "BBB+" or "BBB-"
            average='binary'
        ),
        'Precision': make_scorer(
            precision_score, #Need pos_label
            pos_label='BBB+',
            average='binary'
        ),
        'F1': make_scorer(
            f1_score, #Need pos_label
            pos_label='BBB+',
            average='binary'
        ),
        'Accuracy': 'accuracy', #accuracy_score doesn't need pos_label
        'Balanced accuracy': 'balanced_accuracy',
        'AUROC': 'roc_auc'
    },
    refit='AUROC',

    n_jobs=1,
    verbose=10
)

mlp_classifier_grid_search.fit(classification_X_train, classification_y_train)

end_time = datetime.now()
print('GridSearchCV took {}'.format(end_time - start_time))

Fitting 16 folds for each of 48 candidates, totalling 768 fits
[CV 1/16; 1/48] START activation=relu, alpha=0.001, batch_size=16, hidden_layer_sizes=(5,), learning_rate=constant
[CV 1/16; 1/48] END activation=relu, alpha=0.001, batch_size=16, hidden_layer_sizes=(5,), learning_rate=constant; AUROC: (test=0.910) Accuracy: (test=0.844) Balanced accuracy: (test=0.844) F1: (test=0.852) Precision: (test=0.813) Recall: (test=0.895) total time=   1.2s
[CV 2/16; 1/48] START activation=relu, alpha=0.001, batch_size=16, hidden_layer_sizes=(5,), learning_rate=constant
[CV 2/16; 1/48] END activation=relu, alpha=0.001, batch_size=16, hidden_layer_sizes=(5,), learning_rate=constant; AUROC: (test=0.898) Accuracy: (test=0.830) Balanced accuracy: (test=0.830) F1: (test=0.837) Precision: (test=0.805) Recall: (test=0.871) total time=   1.8s
[CV 3/16; 1/48] START activation=relu, alpha=0.001, batch_size=16, hidden_layer_sizes=(5,), learning_rate=constant
[CV 3/16; 1/48] END activation=relu, alpha=0.001, ba

In [9]:
mlp_classifier_results_df = pd.DataFrame(mlp_classifier_grid_search.cv_results_)
#Make the GridSearch results into a df

mlp_classifier_results_df.drop(
    list(mlp_classifier_results_df.filter(regex='time|split|std')),
    axis=1,
    inplace=True
)  # Remove columns that aren't very interesting
mlp_classifier_results_df = mlp_classifier_results_df.sort_values(
    by='rank_test_AUROC')

mlp_classifier_results_df

,param_activation,param_alpha,param_batch_size,param_hidden_layer_sizes,param_learning_rate,params,mean_test_Recall,rank_test_Recall,mean_test_Precision,rank_test_Precision,mean_test_F1,rank_test_F1,mean_test_Accuracy,rank_test_Accuracy,mean_test_Balanced accuracy,rank_test_Balanced accuracy,mean_test_AUROC,rank_test_AUROC
23,relu,1,64,"(10, 10)",adaptive,"{'activation': 'relu', 'alpha': 1, 'batch_size...",0.881635,29,0.853227,1,0.867009,1,0.864797,1,0.864803,1,0.930046,1
22,relu,1,64,"(10, 10)",constant,"{'activation': 'relu', 'alpha': 1, 'batch_size...",0.881635,29,0.853227,1,0.867009,1,0.864797,1,0.864803,1,0.930046,1
17,relu,1,16,"(10, 10)",adaptive,"{'activation': 'relu', 'alpha': 1, 'batch_size...",0.877007,41,0.843231,9,0.859578,19,0.856639,13,0.856639,13,0.928623,3
16,relu,1,16,"(10, 10)",constant,"{'activation': 'relu', 'alpha': 1, 'batch_size...",0.877007,41,0.843231,9,0.859578,19,0.856639,13,0.856639,13,0.928623,3
35,tanh,0.001,64,"(10, 10)",adaptive,"{'activation': 'tanh', 'alpha': 0.001, 'batch_...",0.879434,33,0.836334,23,0.857250,33,0.853470,33,0.853462,33,0.927051,5
34,tanh,0.001,64,"(10, 10)",constant,"{'activation': 'tanh', 'alpha': 0.001, 'batch_...",0.879434,33,0.836334,23,0.857250,33,0.853470,33,0.853462,33,0.927051,5
46,tanh,1,64,"(10, 10)",constant,"{'activation': 'tanh', 'alpha': 1, 'batch_size...",0.886983,17,0.841421,11,0.863472,5,0.859681,3,0.859672,3,0.925838,7
47,tanh,1,64,"(10, 10)",adaptive,"{'activation': 'tanh', 'alpha': 1, 'batch_size...",0.886983,17,0.841421,11,0.863472,5,0.859681,3,0.859672,3,0.925838,7
14,relu,1,16,"(10,)",constant,"{'activation': 'relu', 'alpha': 1, 'batch_size...",0.882363,25,0.837867,21,0.859390,21,0.855662,21,0.855661,21,0.925502,9
15,relu,1,16,"(10,)",adaptive,"{'activation': 'relu', 'alpha': 1, 'batch_size...",0.882363,25,0.837867,21,0.859390,21,0.855662,21,0.855661,21,0.925502,9


In [10]:
mlp_classifier_results_df.to_csv(
    r'model_grid_search\mlp_classifier_results.csv',
    index=False
)

In [11]:
best_mlp_classifier = mlp_classifier_grid_search.best_estimator_
save_pickle(
    best_mlp_classifier,
    r'model_pickles\best_mlp_classifier.pkl'
)
# To load this best model again, use load_pickle
# (r'model_pickles\mlp_classifier\best_mlp_classifier.pkl')

0